In [298]:
import dill 
dill.dump_session("chr20_CCR_jason.db")

In [ ]:
import dill
dill.load_session("chr20_CCR_jason.db")

# Jason Kunisaki - Quinlan Lab Rotation
## Evaluate CCRs

This notebook contains the script needed to recapitulate CCRs from the first iteration of CCRs [(see manuscript)](https://www.nature.com/articles/s41588-018-0294-6.epdf?author_access_token=N8RkVYcavtplcSSE2KJkNdRgN0jAjWel9jnR3ZoTv0OG-o3UTB17cpoBs8B6XMBCl-5E0ZvpOii0iPl_hRSMGjWfkG4em7gjy95eV4bkTb0AF-E_dj3obeJfaadTja3aj9hUh1xk_BIztWgJScFx8w%3D%3D) and develop new filter parameters to better define CCRs. These parameters include **VAF, SIFT/PolyPhen scores, and coverage**. Use [PathoScore](https://github.com/quinlan-lab/pathoscore) to determine the efficacy of CCRs as a classifier for clinvar pathogenic variants. SIFT and PolyPhen interpretations can be found in the [VEP annotation descriptions](https://uswest.ensembl.org/info/genome/variation/prediction/protein_function.html). VAF filter is based on the percentile of VAFs across all variants from exome and whole genome sequencing (percentiles are calculated independent of sequencing used).   

wget https://storage.googleapis.com/gnomad-public/release/2.1.1/vcf/exomes/gnomad.exomes.r2.1.1.sites.20.vcf.bgz
gunzip -c gnomad.exomes.r2.1.1.sites.20.vcf.bgz > gnomad.exomes.r2.1.1.sites.20.vcf

# Get necessary files 
```
Exome Sequencing Gnomad file: 
cd ~/git/quinlan_lab_rotation/reference/
wget https://storage.googleapis.com/gnomad-public/release/2.0.2/vcf/exomes/gnomad.exomes.r2.0.2.sites.vcf.bgz
gunzip -c gnomad.exomes.r2.0.2.sites.vcf.bgz > gnomad.exomes.r2.0.2.sites.vcf

WGS Gnomad file: 
wget https://storage.googleapis.com/gnomad-public/release/2.0.2/vcf/genomes/gnomad.genomes.r2.0.2.sites.coding_only.chr1-22.vcf.bgz
gunzip -c gnomad.genomes.r2.0.2.sites.coding_only.chr1-22.vcf.bgz > gnomad.genomes.r2.0.2.sites.coding_only.chr1-22.vcf
    
GFF/GTF file: 
wget -P data ftp://ftp.ensembl.org/pub/grch37/current/gff3/homo_sapiens/Homo_sapiens.GRCh37.87.gff3.gz
tabix Homo_sapiens.GRCh37.87.gff3.gz

Exome Sequencing: 
cd ~/git/quinlan_lab_rotation/reference/coverage/exome
for i in {1..22} X Y; do wget https://storage.googleapis.com/gnomad-public/release/2.0.2/coverage/exomes/gnomad.exomes.r2.0.2.chr$i.coverage.txt.gz; done


Whole Genome Sequencing: 
cd ~/git/quinlan_lab_rotation/reference/coverage/wgs/
for i in {1..22} X Y; do wget https://storage.googleapis.com/gnomad-public/release/2.0.2/coverage/genomes/gnomad.genomes.r2.0.2.chr$i.coverage.txt.gz; done
```

## Import modules

In [332]:
import cyvcf2
import sys
import exter ## exter.py file has to be in the working directory 
import re
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import percentileofscore
import subprocess
import os
import fnmatch
import csv
from IPython.display import SVG
from sklearn import preprocessing

## Define the gff and vcf file path 
gff_path = "/Users/jasonkunisaki/git/quinlan_lab_rotation/reference/Homo_sapiens.GRCh37.82.chr20.gff3" 
exome_vcf_path = "/Users/jasonkunisaki/git/quinlan_lab_rotation/reference/gnomad.exomes.r2.0.2.sites.vcf"
wgs_vcf_path = "/Users/jasonkunisaki/git/quinlan_lab_rotation/reference/gnomad.genomes.r2.0.2.sites.coding_only.chr1-22.vcf"
exome_coverage_dir = "~/git/quinlan_lab_rotation/reference/coverage/exome/gnomad.exomes.r2.0.2.chr"
wgs_coverage_dir = "~/git/quinlan_lab_rotation/reference/coverage/exome/gnomad.genomes.r2.0.2.chr"
wgs_vcf_path = exome_vcf_path
wgs_coverage_dir = exome_coverage_dir

## Define SIFT/PolyPhen filter parameters 
sift_polyphen_filter = False
sift_cutoff = 0.05
polyphen_cutoff = 0.908

## Define VAF filter parameters 
vaf_filter = False
vaf_cutoff = 50 ## Percentile to use 

## Assign the pathoscore key name
if sift_polyphen_filter: 
    pathoscore_name = "new_CCR_" + "sift_cutoff_" + str(sift_cutoff) + "_polyphen_cutoff_" + str(polyphen_cutoff)
if not sift_polyphen_filter: 
    pathoscore_name = "new_CCR_no_sift_polyphen_filter"
if vaf_filter: 
    pathoscore_name = pathoscore_name + "_vaf_cutoff_" + str(vaf_cutoff) + "th_percentile"
if not vaf_filter: 
    pathoscore_name = pathoscore_name + "_no_vaf_filter"
print("File name: " + pathoscore_name + ".bed")

## Assign test gene name 
test_key = "KCNQ2"

File name: new_CCR_no_sift_polyphen_filter_no_vaf_filter.bed


## Function that defines functional variants; see [manuscript](https://www.nature.com/articles/s41588-018-0294-6.epdf?author_access_token=N8RkVYcavtplcSSE2KJkNdRgN0jAjWel9jnR3ZoTv0OG-o3UTB17cpoBs8B6XMBCl-5E0ZvpOii0iPl_hRSMGjWfkG4em7gjy95eV4bkTb0AF-E_dj3obeJfaadTja3aj9hUh1xk_BIztWgJScFx8w%3D%3D) for more details 

In [333]:
## Function to identify functional mutations
def isfunctional(csq):
    return any(c in csq['Consequence'] for c in ('stop_gained', 'stop_lost', 'start_lost', 
                                                 'initiator_codon_variant', 'rare_amino_acid_variant', 
                                                 'missense_variant', 'protein_altering_variant', 
                                                 'frameshift_variant', 'inframe_insertion', 'inframe_deletion')) \
    or (('splice_donor_variant' in csq['Consequence'] or 'splice_acceptor_variant' in csq['Consequence']) \
        and 'coding_sequence_variant' in csq['Consequence'])

## Function to read through a VCF file and obtain all variants. This step requires the [cyvcf2](https://github.com/brentp/cyvcf2/tree/master/cyvcf2) function to read and process the vcf file.

In [334]:
def processVCF(vcf_path, gff_path, sift_polyphen_filter, sift_cutoff, polyphen_cutoff, type="exome"):
    ## Read in the VCF file 
    vcf = cyvcf2.VCF(vcf_path)
    
    ## Get the VEP description fields into a 'list'
    kcsq = vcf["vep"]["Description"].split(":")[1].strip(' "').split("|")
    
    ## Read in the gff transcript file and subset calls from chr20
    transcripts = exter.read_gff(gff_path)
    transcripts20 = transcripts["20"] # get chr20

    ## Make an empty dictionary 
    local_by_gene = defaultdict(list)
    
    ## For each variant in the VCF file: 
    for variant in vcf:
        if variant.CHROM != "20": continue ## Remove non-chr20 variants (for now)
        
        ## Remove variants that have "None, PASS, SEGDUP, or LCR" as the filter result
        if not (variant.FILTER is None or variant.FILTER in ["PASS", "SEGDUP", "LCR"]): continue
        
        ## Get the variant 'INFO' field from the VCF file for each variant
        info = variant.INFO
        
        ## Set the functional consequence of the variant to 'False'
        any_functional = False
        
        ## Merge the kcsq description keys with 
        csqs = [dict(zip(kcsq, c.split("|"))) for c in info['vep'].split(",")]

        ## For all variant/consequences that are within protein_coding regions
        for csq in (c for c in csqs if c['BIOTYPE'] == 'protein_coding'):
            ## Ignore intronic variants 
            if csq['Feature'] == '' or csq['EXON'] == '': continue #or csq['cDNA_position'] == '': continue
            
            ## Check to see if the variant's consequence is functional (based on the mutation types listed above)
            if not isfunctional(csq): continue
            
            ## Assigns True to the variant as functional if it is NOT intronic and is deemed as functional by the 'isfunctional' function
            any_functional = True

            ## Get the exonic positions of the variants
            local = transcripts20.localize(variant.start, variant.end)

            ## Get the VAF of the variant
            vaf = variant.INFO.get('AF')

            ## Get the SIFT/PolyPhen results (IS THIS RIGHT)
            SIFT = csq["SIFT"]
            PolyPhen = csq["PolyPhen"]

            ## Get the reference allele 
            ref_allele = variant.REF

            ## Get string for the mutation 
            mut = []
            for alt in variant.ALT: 
                mut.append(ref_allele + ">" + alt) 

            ## Perform SIFT and Polyphen filters to remove the least delterious/damaging variant(s)
            ## Go through each pair of scores and check if the varaint has SIFT and PolyPhen scores         
            if sift_polyphen_filter == True: 
                sift_score = sift_cutoff + 0.1
                polyphen_score = polyphen_cutoff - 0.1
                if re.findall("deleterious", SIFT):
                    sift_score = float(re.split('\)', re.split('\(', SIFT)[1])[0])
                if re.findall("damaging", PolyPhen):
                    polyphen_score = float(re.split('\)', re.split('\(', PolyPhen)[1])[0])
                ## SIFT score has to be below the sift_cutoff to pass 
                ## PolyPhen score has to be above the polyphen_cutoff to pass 
                if not sift_score < sift_cutoff and polyphen_score > polyphen_cutoff:
                    continue
            if sift_polyphen_filter == False: 
                sift_score = SIFT
                polyphen_score = PolyPhen
            
            for l in local:
                l["chr"] = variant.CHROM ## Get the chromosome 
                l["chr_start"] = variant.start
                l["chr_stop"] = variant.end
                l["mutation"] = mut
                l["vaf"] = vaf
                l["SIFT"] = sift_score
                l["PolyPhen"] = polyphen_score
                l["type"] = type
                local_by_gene[l["gene"]].append(l)
            break
    return(local_by_gene)

In [335]:
## Read through gnomad WGS vcf variant file 
wgs_local_by_gene = processVCF(vcf_path=wgs_vcf_path, gff_path=gff_path, sift_polyphen_filter=sift_polyphen_filter, sift_cutoff=sift_cutoff, polyphen_cutoff=polyphen_cutoff, type="whole genome sequencing")

KeyError: b'vep'

In [331]:
## Read through gnomad EXOME vcf variant file
exome_local_by_gene = processVCF(vcf_path=exome_vcf_path, gff_path=gff_path, sift_polyphen_filter=sift_polyphen_filter, sift_cutoff=sift_cutoff, polyphen_cutoff=polyphen_cutoff, type="exome sequencing")

gene = KCNQ2, variant start: 62037999 variant end: 62038000sift: deleterious_low_confidence(0) polyphen: possibly_damaging(0.628)
gene = KCNQ2, variant start: 62038002 variant end: 62038003sift: deleterious_low_confidence(0.01) polyphen: benign(0.392)
gene = KCNQ2, variant start: 62038010 variant end: 62038011sift: deleterious_low_confidence(0.01) polyphen: possibly_damaging(0.891)
gene = KCNQ2, variant start: 62038013 variant end: 62038014sift: tolerated_low_confidence(0.92) polyphen: benign(0.014)
gene = KCNQ2, variant start: 62038021 variant end: 62038022sift: tolerated_low_confidence(0.34) polyphen: benign(0.109)
gene = KCNQ2, variant start: 62038043 variant end: 62038044sift: deleterious(0.01) polyphen: benign(0.22)
gene = KCNQ2, variant start: 62038045 variant end: 62038046sift: deleterious(0) polyphen: probably_damaging(0.956)
gene = KCNQ2, variant start: 62038049 variant end: 62038050sift: tolerated(1) polyphen: benign(0.287)
gene = KCNQ2, variant start: 62038055 variant end: 6

gene = KCNQ2, variant start: 62046388 variant end: 62046389sift: tolerated(0.11) polyphen: benign(0.037)
gene = KCNQ2, variant start: 62046393 variant end: 62046394sift: tolerated(0.37) polyphen: benign(0.037)
gene = KCNQ2, variant start: 62046397 variant end: 62046398sift: deleterious(0.03) polyphen: possibly_damaging(0.847)
gene = KCNQ2, variant start: 62046401 variant end: 62046402sift: tolerated(0.36) polyphen: benign(0.008)
gene = KCNQ2, variant start: 62046402 variant end: 62046403sift: tolerated(0.17) polyphen: benign(0.149)
gene = KCNQ2, variant start: 62046407 variant end: 62046408sift: deleterious(0.03) polyphen: probably_damaging(0.95)
gene = KCNQ2, variant start: 62046408 variant end: 62046409sift: tolerated(0.28) polyphen: probably_damaging(0.923)
gene = KCNQ2, variant start: 62046413 variant end: 62046414sift: tolerated(0.11) polyphen: possibly_damaging(0.571)
gene = KCNQ2, variant start: 62046414 variant end: 62046415sift: deleterious(0.01) polyphen: benign(0.065)
gene =

gene = KCNQ2, variant start: 62078143 variant end: 62078144sift: deleterious(0.04) polyphen: benign(0.006)
gene = KCNQ2, variant start: 62078164 variant end: 62078165sift: deleterious(0) polyphen: benign(0.403)
gene = KCNQ2, variant start: 62078174 variant end: 62078175sift: tolerated(0.53) polyphen: benign(0.002)
gene = KCNQ2, variant start: 62103602 variant end: 62103603sift: tolerated(0.11) polyphen: probably_damaging(0.99)
gene = KCNQ2, variant start: 62103614 variant end: 62103615sift: tolerated(0.85) polyphen: benign(0.005)
gene = KCNQ2, variant start: 62103623 variant end: 62103624sift: tolerated(0.23) polyphen: benign(0.286)
gene = KCNQ2, variant start: 62103626 variant end: 62103627sift: tolerated(0.77) polyphen: benign(0.07)
gene = KCNQ2, variant start: 62103631 variant end: 62103632sift: tolerated(0.36) polyphen: benign(0.198)
gene = KCNQ2, variant start: 62103656 variant end: 62103657sift: deleterious(0.02) polyphen: possibly_damaging(0.771)
gene = KCNQ2, variant start: 621

# Filter variants based on coverage cutoffs 
See [CCR github repo](https://github.com/quinlan-lab/ccr/blob/master/getfiles.sh) for information on the original files used. To pass the coverage filter, greater than **50% of the samples have to have > 10X coverage at the variant's position.** 


```
Exome Sequencing: 
cd ~/git/quinlan_lab_rotation/reference/coverage/exome
for i in {1..22} X Y; do wget -P data https://storage.googleapis.com/gnomad-public/release/2.0.1/coverage/exomes/gnomad.exomes.r2.0.1.chr$i.coverage.txt.gz; done
for i in {1..22} X Y; do wget -P data https://storage.googleapis.com/gnomad-public/release/2.0.1/coverage/exomes/gnomad.exomes.r2.0.1.chr$i.coverage.txt.gz.tbi; done
mv data/* .
rm -rf data  

Whole Genome Sequencing: 
cd ~/git/quinlan_lab_rotation/reference/coverage/wgs/
for i in {1..22} X Y; do wget -P data https://storage.googleapis.com/gnomad-public/release/2.0.1/coverage/genomes/gnomad.genomes.r2.0.1.chr$i.coverage.txt.gz; done
for i in {1..22} X Y; do wget -P data https://storage.googleapis.com/gnomad-public/release/2.0.1/coverage/genomes/gnomad.genomes.r2.0.1.chr$i.coverage.txt.gz.tbi; done
mv data/* .
rm -rf data 
```

In [ ]:
## Function to apply coverage filters 
def coverage_filter(data, coverage_dir): 
    filter_variants = defaultdict(list)
    prev_chr = "temp"
    for key in data.keys(): 
        subset_variants = data[key]
        coverage_variants_per_gene = []
        
        ## Get the chromosome 
        chr = subset_variants[0]["chr"]
  
        ## Read in the coverage data
        if prev_chr != chr: 
            coverage_file = coverage_dir + str(chr) + ".coverage.txt.gz"
            coverage_df = pd.read_csv(coverage_file, sep="\t")
            print("reading in " + coverage_file)
#         else: 
#             coverage_df = coverage_df
        
        ## Go through each variant within each gene and filter based on coverage
        for variant in subset_variants: 
            chr_start = variant["chr_start"]
            chr_end = variant["chr_stop"]
                
            ## Remove sites that do not meet the coverage cutoff 
            variant_coverage = coverage_df[(coverage_df.pos >= chr_start) & 
                                          (coverage_df.pos <= chr_end)]
            coverage_10X = variant_coverage['10'].tolist()
            if not all(i >= .50 for i in coverage_10X): continue
            
            ## Append variant to the coverage_variants_per_gene dataset
            coverage_variants_per_gene.append(variant)
        
        ## Add variants to final dataset 
        filter_variants[key] = coverage_variants_per_gene
        
        prev_chr = chr
    return(filter_variants)    


In [ ]:
cov_filter_df_exome = coverage_filter(data=exome_local_by_gene, coverage_dir=exome_coverage_dir)

In [ ]:
cov_filter_df_wgs = coverage_filter(data=wgs_local_by_gene, coverage_dir=wgs_coverage_dir)

# Get the VAF percentiles to determine appropriate cutoffs
* Use all variants that are above the n-th percentile of VAFs 
* Get the VAF percentiles from exome and whole genome variants separately 

In [ ]:
## Function to obtain list of vaf values from exome and wgs variants 
def get_vaf_values(vcf): 
    vaf_list = []
    for key in vcf.keys(): 
        for variant in vcf[key]: 
            vaf_list.append(variant["vaf"])
    return(vaf_list)

## Function to apply vaf filter on exome and wgs variants 
def apply_vaf_filter(vcf, cutoff):  
    vaf_all_genes = defaultdict(list)
    for key in vcf.keys():
        vaf_by_gene = []
        for variant in vcf[key]: 
            if variant["vaf"] < cutoff: 
                continue
            vaf_by_gene.append(variant)
        vaf_all_genes[key] = vaf_by_gene
    return(vaf_all_genes)

In [ ]:
if not vaf_filter: 
    exome_variants = cov_filter_df_exome
    wgs_variants = cov_filter_df_wgs
if vaf_filter: 
    ## Get the VAF values from exome and wgs samples
    exome_vaf = get_vaf_values(vcf=cov_filter_df_exome)
    exome_vaf = np.array(exome_vaf)
    wgs_vaf = get_vaf_values(vcf=cov_filter_df_wgs)   
    wgs_vaf = np.array(wgs_vaf)
    
    ## Get the numpy percentiles of vaf values 
    exome_vaf_cutoff = np.percentile(exome_vaf, vaf_cutoff)
    wgs_vaf_cutoff = np.percentile(wgs_vaf, vaf_cutoff)
    
    ## Apply vaf filter on exome data 
    exome_variants = apply_vaf_filter(vcf=cov_filter_df_exome, cutoff=exome_vaf_cutoff)
    
    ## Apply vaf filter on wgs data 
    wgs_variants = apply_vaf_filter(vcf=cov_filter_df_wgs, cutoff=wgs_vaf_cutoff)

# Filter variants based on segmental duplication and self-chains 
See [CCR github repo](https://github.com/quinlan-lab/ccr/blob/master/getfiles.sh) for information on the original files used. Creating the bed files requires the process (or a variation of the process) described below: 

```
Segmental Duplications: 
cd ~/git/quinlan_lab_rotation/reference/segmental_duplications
wget ftp://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/genomicSuperDups.txt.gz
zcat < genomicSuperDups.txt.gz | cut -f 2-4 | sed 's/^chr//g' | sort -k1,1 -k2,2n > segmental_duplication_file.bed
## zcat < genomicSuperDups.txt.gz | cut -f 2-4 | sed 's/^chr//g' | sort -k1,1 -k2,2n | bedtools merge | bgzip -c > data/segmental.bed.gz; cd data; tabix -f segmental.bed.gz

Self Chains: 
cd ~/git/quinlan_lab_rotation/reference/self_chains
wget ftp://hgdownload.cse.ucsc.edu/goldenpath/hg19/database/chainSelf.txt.gz
gunzip chainSelf.txt.gz
python get-chain.py | sed 's/^chr//g' | sort -k1,1 -k2,2n > selfchains_final.bed
## python get-chain.py | sed 's/^chr//g' | sort -k1,1 -k2,2n | bedtools merge | bgzip -c > data/self-chains.id90.bed.gz; cd data; tabix -f self-chains.id90.bed.gz
```

In [ ]:
## Read in the segmental duplication and self-chain data 
segdup_data = pd.read_csv("/Users/jasonkunisaki/git/quinlan_lab_rotation/reference/segmental_duplications/segmental_duplication_file.bed", sep='\t', header=None) 
selfchain_data = pd.read_csv("/Users/jasonkunisaki/git/quinlan_lab_rotation/reference/self_chains/selfchains_final.bed", sep='\t')
remove_variants = selfchain_data
remove_variants.append(segdup_data)
remove_variants.columns = ["chr", "start", "end"]

## Function to apply segmental duplication and self-chain filters
def remove_segdup_selfchains(data, remove_variants=remove_variants):
    filter_variants = defaultdict(list)
    for key in data.keys(): 
        subset_variants = data[key] ## Subset variants by gene 
        variants_non_segdup_selfchain_per_gene = []
        for variant in subset_variants: 
            chromosome = variant["chr"]
            chr_start = variant["chr_start"]
            chr_end = variant["chr_stop"]

            ## Subset remove_variants df based on chr
            temp_rm_variants = remove_variants[remove_variants.chr==chromosome]
            temp_rm_variants = temp_rm_variants[(((temp_rm_variants.start <= chr_start) & 
                                                (temp_rm_variants.end >= chr_start)) | ## if the variant's start is within the bad region 
                                               ((temp_rm_variants.start <= chr_end) & 
                                                (temp_rm_variants.end >= chr_end)) | ## if the variant's end is within the bad region
                                                ((temp_rm_variants.start >= chr_start) &
                                                temp_rm_variants.end <= chr_end))] ## if the bad region is within the variant's boundaries

            ## Remove variants if the chr_start/end positions are within the remove_variants windows
            if len(temp_rm_variants) > 0: 
                #print(variant)
                continue 
            if len(temp_rm_variants) == 0: 
                variants_non_segdup_selfchain_per_gene.append(variant)
        
        filter_variants[key] = variants_non_segdup_selfchain_per_gene
    return(filter_variants)

In [ ]:
no_segdup_selfchain_exome = remove_segdup_selfchains(data=exome_variants, remove_variants=remove_variants)

In [ ]:
no_segdup_selfchain_wgs = remove_segdup_selfchains(data=wgs_variants, remove_variants=remove_variants)

# Combine the exome and whole genome variants that passed the above filters: 
* VAF cutoff 
* SIFT/PolyPhen scores 
* Remove segmental duplications and self-chains 
* Coverage 

In [ ]:
## Combine wgs and exome variant data. Duplicate variants will be accounted for below. 
combined_wgs_exome_variants = defaultdict(list)
for key in no_segdup_selfchain_wgs.keys(): 
    if len(no_segdup_selfchain_exome[key]) == 0: 
        combined_wgs_exome_variants[key] = no_segdup_selfchain_wgs[key]
    else: 
        combined_wgs_exome_variants[key] = no_segdup_selfchain_exome[key] + no_segdup_selfchain_wgs[key]

In [ ]:
combined_wgs_exome_variants[test_key]

## Perform quality check on pathogenic variants. Remove genes with no variants after applying filters from above. 

In [ ]:
final_variant_vcf = defaultdict(list)
## Check if there is at least one variant in each gene 
for key in combined_wgs_exome_variants.keys():
    if len(combined_wgs_exome_variants[key]) > 0:
        final_variant_vcf[key] = combined_wgs_exome_variants[key]

## Fix overlapping variants within each gene + apply SIFT/PolyPhen and VAF filters
| Position |   V1   |   V2   |   V3   |   V4   |   V5   |   V6   |   V7   |   V8   |   V9   |  V10   |  V11   |
| :------- | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: |
| Start (0-based)    | 126244 | 126244 | 126253 | 126255 | 126258 | 126258 | 126260 | 126264 | 126264 | 126265 | **126279** |
| End (1-based)      | 126245 | 126254 | 126254 | 126256 | 126259 | 126262 | **126279** | 126265 | 126265 | 126266 | 126280 |

### Combine variant positions for:
+ adjacent bases: **V7 ends at 126279 (1-based) and V11 starts at 126279 (0-based). The variant windows are combined.**
+ equivalent variants (in terms of position) 
+ overlapping variants

### The determined variant range should encompass all of the above variants
| Position | window 1 | window 2 | window 3 | 
| :------: | :------: | :------: | :------: |
| Start    | 126244   | 126255   | 126258   |
| End      | 126254   | 126256   | 126280   |

In [ ]:
## Inititalize the master list 
unique_variant_pos_all_genes = defaultdict(list)
for key in final_variant_vcf.keys():
    subset_variants = final_variant_vcf[key]
    ## Get all of the start and stop variant positions within the gene
    start_positions = [d["chr_start"] for d in subset_variants]
    end_positions = [d["chr_stop"] for d in subset_variants]
    exonic_end_positions = [d["stop"] for d in subset_variants]

    ## Go throouh each start/end position and check if there are overlapping variants 
    unique_variant_pos_per_gene = []
    positions = []
    for start, end in zip(start_positions, end_positions):
        ## Get variants in which the start is within the variant's start/end
        temp1 = list(filter(lambda subset_variants: 
               subset_variants["chr_start"] <= start and subset_variants["chr_stop"] >= start, subset_variants))
        
        ## Check to see if there are variants that overlap with maximum variant's end position from the above subset
        new_end = max([d["chr_stop"] for d in temp1])
        temp2 = list(filter(lambda subset_variants:
                   subset_variants["chr_start"] <= new_end and subset_variants["chr_stop"] >= new_end, subset_variants))
        
        ## Combine all variants that overlap with the intial variant(s) in question
        for v in temp2: 
            if v not in temp1: 
                temp1.append(v)

        ## Get the index at which an adjacent variant's start position overlaps with the end position of the adjusted window
        ## Assign the maximum end pos of the adjacent variant as the window's stop position
        while max(set([d["chr_stop"] for d in temp1])) in start_positions and \
        max(set([d["chr_stop"] for d in temp1])) not in positions: ## Make sure it does not pull a variant that was already accounted for in a window
            d = defaultdict(list)
            for index, e in enumerate(start_positions):
                d[e].append(index)
            
            ## Get all the end positions of adjacent variants 
            end_index = d[max(set([d["chr_stop"] for d in temp1]))]
            
            ## Replace the maximum of the adjusted window with the adjacent variant's stop
            temp1[len(temp1)-1]["stop"] = max(list(np.array(exonic_end_positions)[end_index]))
            temp1[len(temp1)-1]["chr_stop"] = max(list(np.array(end_positions)[end_index]))

        ## If the variant is NOT the first variant 
        if len(positions) > 0: 
            ## Check to see if the variant's start and stop falls in a window already calculated 
            prev_start_positions = [d["final_start_positions"] for d in positions] ## Get start positions
            prev_end_positions = [d["final_end_positions"] for d in positions] ## Get end positions
            for s, e in zip(prev_start_positions, prev_end_positions):
                ## If the variant' start or end falls within a previously calculated window, skip the variant 
                if (start >= s and start <= e) or (end >= s and start <= e):
                    temp1 = []
                    break

        ## If there are overlapping variants
        if len(temp1) > 1: 
            final_temp = {
                "gene": key,
                "start": min(set([d["start"] for d in temp1])),
                "stop": max(set([d["stop"] for d in temp1])),
                "strand": temp1[1]["strand"],
                "exon": temp1[1]["exon"],
                "chr": temp1[1]["chr"],
                "chr_start": min(set([d["chr_start"] for d in temp1])),
                "chr_stop": max(set([d["chr_stop"] for d in temp1])),
                "mutation": "overlapping mutations",
                "vaf": temp1[1]["vaf"],
                "SIFT": temp1[1]["SIFT"],
                "PolyPhen": temp1[1]["PolyPhen"]
            }
            unique_variant_pos_per_gene.append(final_temp)

        ## If there are no overlapping variants 
        if len(temp1) == 1:
            final_temp = temp1[0]
            unique_variant_pos_per_gene.append(final_temp)

        ## Get the complete start and stop positions for variants within gene
        test = {
        "final_start_positions": final_temp["chr_start"], ## 0-based
        "final_end_positions": final_temp["chr_stop"] ## 1-based 
        } 

        ## Store the start and stop positions for each adjusted variant range
        if len(temp1) > 0:
            positions.append(test)
    
    ## Sort each variant within each gene based on the genomic start position
    unique_variant_pos_per_gene = sorted(unique_variant_pos_per_gene, key = lambda i: i["chr_start"])
    unique_variant_pos_all_genes[key] = unique_variant_pos_per_gene

In [ ]:
unique_variant_pos_all_genes[test_key]

## Caclulate CCR window sizes and positions based on variant coordinates
```
   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20    ## 1 base
 | T | T | G | A | T | C | G | A | T | A | G | C | T | A | C | C | G | A | T | C |
 0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  ## 0 base 
```
**Examples**
+ **1 base b/w variants:** 2 *(1-based; variant_plus_one.start; 2-3 G)* - 1 *(0-based; variant.end; 0-1 T)* = 1
+ **Multiple bases b/w variants**: 10 *(0-based; variant_plus_one.start; 10-11 G)* - 4 *(1-based; variant.end; 0-4 TTGA)* = 6

In [ ]:
## Store the genes as keys 
## Store the window sizes/variant positions in dictionaries  
final_window_dataset = {}
for key in unique_variant_pos_all_genes.keys(): 
    ## Subset dictionaries (i.e. list of variants) based on each key (i.e. each gene)
    subset_variants = unique_variant_pos_all_genes[key]
    
    ## Go through the variants within each gene and measure the CCR window size 
    subset_window_list = []
    
    ## Go through each variant, get the variant pair, and then calculate window size 
    for variant, variant_plus_one in zip(subset_variants, subset_variants[1:]):
        ## Calculate window position of the variant
        variant1_window = {"gene": key,
        "chromosome": "chr" + variant["chr"],
        "variant1_start": variant["start"],
        "variant1_stop": variant["stop"],
        "variant2_start": "NA",
        "variant2_stop": "NA",
        "variant1_vaf": variant["vaf"],
        "variant2_vaf": "NA",
        "variant1_mut": variant["mutation"],
        "variant2_mut": "NA",
        "variant1_SIFT": variant["SIFT"],
        "variant1_PolyPhen": variant["PolyPhen"],
        "variant2_SIFT": "NA",
        "variant2_PolyPhen": "NA",
        "window_size": 0,
        "window_start": variant["chr_start"],
        "window_stop": variant["chr_stop"],
        "varflag": "VARTRUE"
        }

        ## Calculate window positions between variants
        window_dict = {"gene": key,
        "chromosome": "chr" + variant["chr"],
        "variant1_start": variant["start"],
        "variant1_stop": variant["stop"],
        "variant2_start": variant_plus_one["start"],
        "variant2_stop": variant_plus_one["stop"],
        "variant1_vaf": variant["vaf"],
        "variant2_vaf": variant_plus_one["vaf"],
        "variant1_mut": variant["mutation"],
        "variant2_mut": variant_plus_one["mutation"],
        "variant1_SIFT": "NA",
        "variant1_PolyPhen": "NA",
        "variant2_SIFT": "NA",
        "variant2_PolyPhen": "NA",
        "window_size": max(0, variant_plus_one["start"] - variant["stop"]),
        "window_start": variant["chr_stop"],
        "window_stop": variant_plus_one["chr_start"], 
        "varflag": "VARFALSE"
        }

        ## Append window for variant and adjacent variants
        subset_window_list.append(variant1_window)
        if window_dict["window_size"] > 0 :
            subset_window_list.append(window_dict)

    ## Get the information for the last variant in the GOI 
    last_variant = subset_variants[len(subset_variants)-1]
    last_variant = {"gene": key,
        "chromosome": "chr" + last_variant["chr"],
        "variant1_start": last_variant["start"],
        "variant1_stop": last_variant["stop"],
        "variant2_start": "NA",
        "variant2_stop": "NA",
        "variant1_vaf": last_variant["vaf"],
        "variant2_vaf": "NA",
        "variant1_mut": last_variant["mutation"],
        "variant2_mut": "NA",
        "variant1_SIFT": last_variant["SIFT"],
        "variant1_PolyPhen": last_variant["PolyPhen"],
        "variant2_SIFT": "NA",
        "variant2_PolyPhen": "NA",
        "window_size": 0,
        "window_start": last_variant["chr_start"],
        "window_stop": last_variant["chr_stop"],
        "varflag": "VARTRUE"
        }
    
    ## Append window for the last variant 
    subset_window_list.append(last_variant)

    ## Append each gene's window data to the associated gene in the final dataset
    assert(key not in final_window_dataset)
    final_window_dataset[key] = subset_window_list

In [ ]:
final_window_dataset[test_key]

# Obtain the percentiles for each CCR window 
## Calculate percentiles based on varflag == VARFALSE value 

In [ ]:
## Define the 'windows' list  
windows = []

## Go through each gene and get the window sizes
for key in final_window_dataset.keys():
    subset_variants = final_window_dataset[key]
    for variant in subset_variants:
#         if variant["varflag"] == "VARFALSE":
#             windows.append(variant["window_size"])
          windows.append(variant["window_size"])

## Sort the 'windows' list
windows.sort()

## Convert windows to numpy array
windows = np.array(windows)

## Go through each of the variants and obtain the scores 
final_dataset = defaultdict(list)
for key in final_window_dataset.keys():
    ## Get the variants found within each gene
    subset_variants = final_window_dataset[key]
    
    ## Define a percentiles list with each window's percentile + other information as the dictionary
    percentiles_single_gene = []
    for variant in subset_variants:
        ## If the variant is a variant, assign percentile as 0
        if variant["varflag"] == "VARTRUE": 
            variant["percentile"] = 0
        
        ## If the variant is a CCR window, assign percentile based on window values 
        if variant["varflag"] == "VARFALSE":        
            variant["percentile"] = round(percentileofscore(windows, variant["window_size"]), 2)
        
        ## Append the dictionary to the list of percentiles for each gene
        percentiles_single_gene.append(variant)
    
    ## Appened all percentiles across all genes to the final dataset list 
    final_dataset[key] = percentiles_single_gene

In [ ]:
final_dataset[test_key]

# Get the weighted percentiles 
Start at 100 (the most constrained region) and scale down proportionally for each subsequent region by the fraction of the length of protein-coding exome already covered by the preceding region. More information can be found [here](https://quinlan-lab.github.io/ccr/).

In [ ]:
## Use panda dataframes to make BED files for PathoScore
new_CCR = pd.DataFrame()
for key in final_dataset.keys():
    gene_pandaDf = pd.DataFrame(final_dataset[key], columns=["chromosome", "window_start", 
        "window_stop", "gene", 
        "window_size", "varflag", "variant1_vaf", "variant1_SIFT", "variant1_PolyPhen", "percentile"])
    new_CCR = pd.concat([new_CCR, gene_pandaDf])

## Change the column names of the combined panda dataframes and remove window_size column
new_CCR.columns = ["#chrom", "start", "end", "gene", "window_size", "varflag",
                   "vaf", "SIFT", "PolyPhen", "residual_pct"]

## Sort (descending) the panda dataframe by residual percentile 
new_CCR = new_CCR.sort_values(by = ["residual_pct", "#chrom", "start"], ascending = False)

## Write the output file and read it back in to remove duplicate index values in pandas dataset 
output_file = pathoscore_name + ".bed" 
new_CCR.to_csv(output_file, sep="\t", index=False)
new_CCR = pd.read_csv(pathoscore_name + ".bed", sep="\t")

## Print the head of new_CCR dataset
new_CCR.head()

In [ ]:
## Get the total length covered by CCRs
totlength = sum(new_CCR["window_size"].tolist())

## Go throuh each row and determine the weighted percentile 
pct = 100.0
opct = new_CCR.loc[0, "residual_pct"]
weighted = [] ## Initialize the weighted percentile list 
for r in new_CCR.itertuples():
    ## Do not perform this calculation on variants
    if r[6] != "VARTRUE":  
        regionlength = int(r[5]) ## Get the CCR size 
        residual_pct = r[len(r)-1] ## Get the residual CCR percentile 
        
        ## If the percentile is not equal to the previous CCR's percentile
        if r[len(r)-1] != opct: 
            pct -= regionlength/totlength*100 ## Perform the adjustment calculation 
            opct = residual_pct ## Assign new CCR percentile
                             
        ## Add weighted percentile to the weighted list 
        weighted.append(pct)
    else: ## Assign 0th percentile to variants 
        weighted.append(0)
        
## TODO: check to see if these percentiles are the final weighted percentiles 
X_train = np.array(weighted).reshape(len(weighted), 1)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,100))
final_pctile = min_max_scaler.fit_transform(X_train)
final_pctile_df = pd.DataFrame(final_pctile)

## Add the weighted CCR values to the panda datase 
weighted_CCR = pd.concat([new_CCR, final_pctile_df], axis=1)
weighted_CCR.columns = ["#chrom", "start", "end", "gene", "window_size", "varflag",
                   "vaf", "SIFT", "PolyPhen", "residual_pct", "ccr_pct"]

weighted_CCR.head()

## Create a BED file to run in PathoScore

In [ ]:
## Sort by position
weighted_CCR = weighted_CCR.sort_values(by=["#chrom", "start", "end"])

## Write the output file and read it back in to remove duplicate index values in pandas dataset 
output_file = pathoscore_name + ".bed" 
weighted_CCR.to_csv(output_file, sep="\t", index=False) ## Write the bed file 
weighted_CCR.to_csv((pathoscore_name + ".txt"), sep="\t", index=False) ## Write the text file 

# Generate the script needed to run the PathoScore function on all bed files present in the directory 

Step 1: create the scripts needed to run PathoScore 

Step 2: bgzip and tabix all of the bed files present in the directory (TODO)

Step 3: annotate benign and pathogenic variants from all bed files 

Step 4: run the PathoScore evaluate function on annotated benign and pathogenic files 

## Write the PathoScore scripts based on the bed files present in the current working directory

In [ ]:
list_of_files = os.listdir(".")
pattern = "*.bed"
## Create a list of bed files there associated column numbers 
bedfile_colnum_list = []
for file in list_of_files: 
    if fnmatch.fnmatch(file, pattern):
        data = pd.read_csv(file, sep='\t')
        colnames = list(data.columns)
        bedfile_colnum_list.append({
            "filename": "~/git/quinlan_lab_rotation/" + file + ".gz",
            "filename_no_suffix": file.split(".bed")[0],
            "colnumber": colnames.index("ccr_pct") + 1
        })

## Use the bedfile names and column numbers to generate pathoscore scripts
score_script = []
evaluate_script = []
for i in range(len(bedfile_colnum_list)): 
    filename = bedfile_colnum_list[i]["filename"]
    filename_no_suffix = bedfile_colnum_list[i]["filename_no_suffix"]
    colnum = str(bedfile_colnum_list[i]["colnumber"])
    score_script.append("--scores " + filename + ":" + filename_no_suffix + ":" + colnum + ":max")
    evaluate_script.append("-s " + filename_no_suffix)
separator = " "
benign_score_script = "python pathoscore.py annotate --prefix benign " + separator.join(score_script) + " ~/git/quinlan_lab_rotation/pathoscore/truth-sets/GRCh37/clinvar/clinvar-benign.20170905.vcf.gz"
pathogenic_score_script = "python pathoscore.py annotate --prefix pathogenic " + separator.join(score_script) + " ~/git/quinlan_lab_rotation/pathoscore/truth-sets/GRCh37/clinvar/clinvar-pathogenic.20170905.vcf.gz"
pathoscore_evaluate_script = "python pathoscore.py evaluate " + separator.join(evaluate_script) + " pathogenic.vcf.gz benign.vcf.gz"

In [ ]:
print(benign_score_script)
print(pathogenic_score_script)
print(pathoscore_evaluate_script)

In [ ]:
file_object = open(r"pathoscore_scripts.sh", "w+")
file_object.write("%s\n%s\n%s\n" % (benign_score_script, pathogenic_score_script, pathoscore_evaluate_script))
file_object.close()

## Bgzip all of the bed files in the directory 

In [ ]:
%%bash 
bash /Users/jasonkunisaki/git/quinlan_lab_rotation/scripts/bgzip.sh /Users/jasonkunisaki/git/quinlan_lab_rotation

## Run the PathoScore funtions that were generated above

In [ ]:
%%bash 
bash /Users/jasonkunisaki/git/quinlan_lab_rotation/pathoscore_scripts.sh

## Move PathoScore output files to the appropriate output directory 

In [ ]:
%%bash 
bash /Users/jasonkunisaki/git/quinlan_lab_rotation/scripts/move_files.sh /Users/jasonkunisaki/git/quinlan_lab_rotation/pathoscore_results/

# Print ROC curve from the PathoScore results

In [ ]:
SVG('/Users/jasonkunisaki/git/quinlan_lab_rotation/pathoscore_results/pathoscore.roc.svg') 